In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from datetime import datetime

driver = uc.Chrome(headless=False, use_subprocess=False)


hello


In [58]:
# get all hrefs that contain https://lottery.broadwaydirect.com/show/
def get_all_show_urls(driver):
    driver.get('https://lottery.broadwaydirect.com/')
    shows = driver.find_elements(By.XPATH, '//a[contains(@href, "https://lottery.broadwaydirect.com/show/")]')
    show_hrefs = list(set([show.get_attribute('href') for show in shows]))
    return show_hrefs

In [66]:
def get_show_entries(driver):
    table = driver.find_element(By.TAG_NAME, "table")
    trs = table.find_elements(By.TAG_NAME, "tr")
    target_hrefs = []
    for tr in trs:
        show_href = None
        # get element inside tr that contains "time" in class
        tds = tr.find_elements(By.TAG_NAME, "td")
        for td in tds:
            if "time" in td.get_attribute("class"):
                show_time = td.text.strip()
                datetime_object = datetime.strptime(show_time, '%m/%d/%y %I:%M %p')
                day_of_week = datetime_object.strftime('%A')
                
            # check for href in td
            if td.find_elements(By.TAG_NAME, "a"):
                show_href = td.find_element(By.TAG_NAME, "a").get_attribute("href")
        
        if show_href is None or 'enter-lottery' not in show_href:
            continue
        
        # filter logic if the showtime isn't appropriate
        if day_of_week in ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]:
            target_hrefs.append(show_href)

    return target_hrefs

In [60]:
show_hrefs = get_all_show_urls(driver)

In [62]:
driver.get(show_hrefs[0])

In [67]:
target_hrefs = get_show_entries(driver)

In [69]:
driver.get(target_hrefs[0])

In [ ]:
def fill_entry(driver):
    

In [85]:
# dlslot_name_first
input_first_name = driver.find_element(By.ID, "dlslot_name_first")

# dlslot_name_last
input_last_name = driver.find_element(By.ID, "dlslot_name_last")

# dlslot_ticket_qty
select_ticket_qty = driver.find_element(By.ID, "dlslot_ticket_qty")

# dlslot_email
input_email = driver.find_element(By.ID, "dlslot_email")

# dlslot_dob_month
input_dob_month = driver.find_element(By.ID, "dlslot_dob_month")
# dlslot_dob_day
input_dob_day = driver.find_element(By.ID, "dlslot_dob_day")
# dlslot_dob_year
input_dob_year = driver.find_element(By.ID, "dlslot_dob_year")

# dlslot_zip
input_zip = driver.find_element(By.ID, "dlslot_zip")

# dlslot_country
select_country = driver.find_element(By.ID, "dlslot_country")

# dlslot_agree
checkbox_agree = driver.find_element(By.ID, "dlslot_agree")

In [86]:
input_first_name.send_keys("John")
input_last_name.send_keys("Doe")
select_ticket_qty.send_keys("2")
input_email.send_keys("john.doe@gmail.com")
input_dob_month.send_keys("01")
input_dob_day.send_keys("01")
input_dob_year.send_keys("1990")
input_zip.send_keys("10001")
select_country.send_keys("USA")

In [ ]:
# find the element ::after and click on it
checkbox_agree.find_element(By.LABEL, "::after").click()

AttributeError: type object 'By' has no attribute 'LabelEntry'